In [ ]:
import os
import pandas as pd
import requests
import xmltodict

API_KEY = 'WS77-PWJ1-MKS5'

In [ ]:
#Reverse lookup a single phone number:
def reverse_lookup(phone_number, column_name, apikey):

    #Create an empty dictionary:
    info_dict = {}

    #Call api with given parameters and save xml as dictionary:
    base_url = "https://ws.serviceobjects.com/gppl2/api.svc/GetPhoneInfo?"
    parameters = {'PhoneNumber': phone_number, 'LicenseKey': apikey}
    response = requests.get(base_url, params=parameters)
    results = xmltodict.parse(response.content)

    #Select phone info section of resulting dictionary:
    phone_results = results["PhoneInfoResponse"]["PhoneInfo"]

    #Save phone number:
    info_dict[column_name] = phone_number

    #Save relevant values in results in smaller dictionary. Save null value if errors:
    try:
        info_dict["Name"] = phone_results["Contacts"]['Contact']["Name"]
    except:
        info_dict["Name"] = 'N/A'
    try:
        info_dict["Address"] = phone_results["Contacts"]['Contact']["Address"]
    except:
        info_dict["Address"] = 'None'
    try:
        info_dict["City"] = phone_results["Contacts"]['Contact']["City"]
    except:
        info_dict["City"] = 'N/A'
    try:
        info_dict["State"] = phone_results["Contacts"]['Contact']["State"]
    except:
        info_dict["State"] = 'N/A'
    try:
        info_dict["PhoneType"] = phone_results["Contacts"]['Contact']["PhoneType"]
    except:
        info_dict["PhoneType"] = 'N/A'
    try:
        info_dict["QualityScore"] = phone_results["Contacts"]['Contact']["QualityScore"]
    except:
        info_dict["QualityScore"] = 'N/A'
    try:
        info_dict["Date"] = phone_results["DateOfPorting"]
    except:
        info_dict["Date"] = 'none'
    try:
        info_dict["Notes"] = phone_results["NoteDescriptions"]
    except:
        info_dict["Notes"] = 'None'
    try:
        info_dict["Provider"] = phone_results['Provider']['Name']
    except:
        info_dict["Provider"] = 'N/A'

    #Return resulting dictionary and full dictionary:
    return(info_dict, results)


#Test all number in a dataframe:
def test_numbers(dataframe, column_name, apikey):

    #Create two empty lists:
    results_dict_list = []
    fun_massive_list = []

    #Iterate through dataframe and call reverse_lookup on each number:
    for row in dataframe.itertuples():
        new_dict = {}
        phone = row.TELEPHONE_NUMBER
        try:
            new_dict, phone_results = reverse_lookup(phone, column_name, apikey)
            results_dict_list.append(new_dict)
            fun_massive_list.append(phone_results)
        except:
            pass
        print(phone_results)

    #Save lists of dictionaries as dataframes:
    new_df = pd.DataFrame(results_dict_list)
    final_df = pd.merge(new_df, dataframe, on=column_name)

    #Return dataframes:
    return(fun_massive_list, final_df)

In [ ]:
sample = pd.read_excel('../../Data/Triangulation/2020-12-23_VT_Verification_Sample.xlsx')
sample.head()

In [ ]:
fun_massive_list, final_df = test_numbers(phone_df, column_header, key)

In [ ]:
INFO, RESULTS = reverse_lookup(sample.TELEPHONE_NUMBER[1], 'TELEPHONE_NUMBER', API_KEY)

In [ ]:
base_url = "https://ws.serviceobjects.com/gppl2/api.svc/GetPhoneInfo?"
parameters = {'PhoneNumber': sample.TELEPHONE_NUMBER[0], 'LicenseKey': API_KEY}
response = requests.get(base_url, params=parameters)
results = xmltodict.parse(response.content)
# phone_results = results["PhoneInfoResponse"]["PhoneInfo"]

In [ ]:
xmltodict.parse(response.content)

In [ ]:
INFO

In [ ]:
RESULTS['PhoneInfo']

In [ ]:
phone_df = sample[2:500]

In [ ]:
fun_massive_list_2, final_df_2 = test_numbers(new_phone_df, 'TELEPHONE_NUMBER', API_KEY)

In [ ]:
final_df_2.to_csv('../../Data/Triangulation/GetPhone_VT_2021_01_11.csv', index=False)

In [ ]:
with open(f'../../Data/Triangulation/GetPhone_VT_RAW_2021_01_11.txt', 'w') as outfile:
        json.dump(fun_massive_list_2, outfile)

In [ ]:
import json

In [ ]:
sample

In [ ]:
completed_phone_numbers = list(phone_df.drop_duplicates('TELEPHONE_NUMBER')['TELEPHONE_NUMBER'])

In [ ]:
new_phone_df = sample[sample.TELEPHONE_NUMBER.isin(completed_phone_numbers)==False].drop_duplicates('TELEPHONE_NUMBER')

In [ ]:
len(sample)

In [ ]:
len(new_phone_df)

In [ ]:
completed_phones = list(set(sample.TELEPHONE_NUMBER))

In [ ]:
roster_phones = pd.read_csv('../../Data/Triangulation/Roster_Phones.csv')

In [ ]:
fun_massive_list_3, final_df_3 = test_numbers(roster_phones, 'TELEPHONE_NUMBER', API_KEY)

In [ ]:
final_df_3.to_csv('../../Data/Triangulation/GetPhone_Rosters_2021_01_13.csv', index=False)
with open(f'../../Data/Triangulation/GetPhone_Rosters_RAW_2021_01_13.txt', 'w') as outfile:
        json.dump(fun_massive_list_3, outfile)

In [ ]:
len(final_df_3)

In [ ]:
roster_phones_2 = pd.read_csv('../../Data/Triangulation/Updated_Roster_Phones.csv')

In [ ]:
roster_2 = roster_phones_2[roster_phones_2['TELEPHONE_NUMBER'].isin(roster_phones.TELEPHONE_NUMBER)==False].dropna()

In [ ]:
fun_massive_list_4, final_df_4 = test_numbers(roster_2[680:], 'TELEPHONE_NUMBER', API_KEY)

In [ ]:
roster_2[680:]

In [ ]:
fun_massive_list_5, final_df_5 = test_numbers(roster_2[680:], 'TELEPHONE_NUMBER', API_KEY)

In [ ]:
import json
final_df_5.to_csv('../../Data/Triangulation/GetPhone_Rosters_2_2021_01_13.csv', index=False)
with open(f'../../Data/Triangulation/GetPhone_Rosters_RAW_2_2021_01_13.txt', 'w') as outfile:
        json.dump(fun_massive_list_5, outfile)

In [ ]:
len(roster_phones_2)

In [ ]:
48420 - (len(sample.drop_duplicates('TELEPHONE_NUMBER'))+len(roster_phones_2))

In [ ]:
ppd = pd.read_csv('../../Data/PPD/ppd_data_20210109.csv')

In [ ]:
ppd_sample = ppd.drop_duplicates('TELEPHONE_NUMBER').sample(8000)

In [ ]:
ppd_sample_2 = ppd[ppd.TELEPHONE_NUMBER.isin(ppd_sample.TELEPHONE_NUMBER)==False].drop_duplicates('TELEPHONE_NUMBER').sample(521)

In [ ]:
fun_massive_list_6, final_df_6 = test_numbers(ppd_sample, 'TELEPHONE_NUMBER', API_KEY)

In [ ]:
ppd_sample_2.to_csv('../../Data/ppd_sample_2.csv', index=False)
fun_massive_list_7, final_df_7 = test_numbers(ppd_sample_2, 'TELEPHONE_NUMBER', API_KEY)

In [ ]:
final_df_7.to_csv('../../Data/Triangulation/GetPhone_PPD_3_2021_01_14.csv', index=False)
with open(f'../../Data/Triangulation/GetPhone_PPD_RAW_3_2021_01_14.txt', 'w') as outfile:
        json.dump(fun_massive_list_7, outfile)

In [ ]:
ppd_sample.to_csv('../../Data/ppd_sampl.csv', index=False)

In [ ]:
len(final_df_6)

In [ ]:
len(fun_massive_list_6)

In [ ]:
final_df_6.to_csv('../../Data/Triangulation/GetPhone_PPD_2_2021_01_13.csv', index=False)
with open(f'../../Data/Triangulation/GetPhone_PPD_RAW_2_2021_01_13.txt', 'w') as outfile:
        json.dump(fun_massive_list_6, outfile)

In [ ]:
ppd[ppd.ADDRESS_TYPE.isin([2.0,3.0])][['MAILING_NAME','MAILING_LINE_1','MAILING_LINE_2','CITY','STATE','ADDRESS_TYPE']].head(20)

In [ ]:
ppd.columns